<a href="https://colab.research.google.com/github/daanbee/tobigs-22nd/blob/main/%ED%88%AC%EB%B9%85%EC%8A%A4_Week5_TS_advance(energy).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Q. 기상 실측 데이터를 토대로 태양광 발전량인 amount(2023-10-15) 값을 예측해보시오.

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from torch.utils.data import TensorDataset, DataLoader

pd.set_option('display.max_column', 20)
pd.set_option('display.max_row', 100)

from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/MyDrive/energy_tobigs_question.csv')
data['time'] = pd.to_datetime(data['time'])

# 피처와 타겟 변수 설정
features = ['cloud', 'temp', 'humidity', 'ground_press', 'wind_speed', 'wind_dir', 'rain', 'snow', 'dew_point', 'vis', 'uv_idx', 'azimuth', 'elevation']
target = 'amount'

# 데이터셋 분리 (2023-10-14까지 학습, 2023-10-15 하루 예측)
train_data = data[data['time'] < '2023-10-15'].copy()
test_data = data[(data['time'] >= '2023-10-15') & (data['time'] < '2023-10-16')].copy()

# 학습 데이터에 NaN 값 제거
train_data = train_data.dropna()

# 데이터 스케일링
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

train_data.loc[:, features] = scaler_x.fit_transform(train_data[features])
train_data.loc[:, target] = scaler_y.fit_transform(train_data[[target]])

test_data.loc[:, features] = scaler_x.transform(test_data[features])

Mounted at /content/drive


### 시퀀스 데이터 생성 함수 작성

In [4]:
# 텐서로 변환해서 텐서데이터셋을 생성하세요. (1~6 번에 내용을 채워보세요.)

# 시퀀스 데이터 생성 함수
# dataX: 시퀀스 데이터를 담은 리스트 (예: [[시퀀스1], [시퀀스2], ...])
# dataY: 각 시퀀스에 대응하는 타겟 값 (예: 발전량)

def build_dataset(time_series, seq_length):
    dataX, dataY = [], []
    for i in range(len(time_series) - seq_length):
        dataX.append(time_series[i:i+seq_length, :-1])
        dataY.append(time_series[i+seq_length, [-1]])
    return np.array(dataX), np.array(dataY)

# 하이퍼파라미터 설정
seq_length = 24
batch_size = 32

# 학습 데이터 생성
# trainX: 24시간의 피처 데이터
# trainY: 그 다음 시간에 대한 발전량 값
trainX, trainY = build_dataset(train_data[features + [target]].values, seq_length)

# 텐서로 변환
trainX_tensor = torch.FloatTensor(trainX)
trainY_tensor = torch.FloatTensor(trainY)

# 텐서데이터셋 생성
train_dataset = TensorDataset(trainX_tensor, trainY_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

### 학습한 시계열 모델을 사용해서 예측을 해보세요 (RNN, LSTM, Transformer, Informer 중에서 선택적으로 활용)

In [ ]:
# 슬라이딩 윈도우 방식으로 2023-10-15의 24시간 발전량 예측해보기 (Direct Multi-step Forecast Strategy 혹은 Recursive Multi-step Forecast 등 이외의 방법론도 자유롭게 사용 가능)

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # 초기 hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # 초기 cell state

        # LSTM forward pass
        out, _ = self.lstm(x, (h0, c0))

        # 마지막 LSTM 출력만 사용 (시퀀스 마지막 시간의 출력)
        out = self.fc(out[:, -1, :])
        return out

# 하이퍼파라미터 설정
input_size = len(features)  # 입력 피처의 개수
hidden_size = 64  # LSTM hidden state 크기
num_layers = 2  # LSTM 레이어 개수
output_size = 1  # 출력 (예측할 타겟 값)
learning_rate = 0.001
num_epochs = 50
seq_length = 24  # 슬라이딩 윈도우 길이

# 모델 초기화
model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(torch.device('cpu'))

# 손실 함수 및 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 모델 학습 (이전 코드에서 진행된 학습 부분을 사용)
for epoch in range(num_epochs):
    model.train()
    for i, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(torch.device('cpu'))
        targets = targets.to(torch.device('cpu'))

        # 예측
        outputs = model(inputs)

        # 손실 계산
        loss = criterion(outputs, targets)

        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


# Recursive Multi-step Forecast for 2023-10-15 (슬라이딩 윈도우 방식)
def recursive_forecast(model, initial_sequence, forecast_steps):
    model.eval()  # 예측 모드로 전환
    predictions = []
    current_sequence = initial_sequence  # 처음에 주어진 시퀀스 (24시간)

    with torch.no_grad():
        for step in range(forecast_steps):
            # 시퀀스를 텐서로 변환
            input_tensor = torch.FloatTensor(current_sequence).unsqueeze(0).to(torch.device('cpu'))

            # 모델로 예측 수행
            predicted = model(input_tensor).item()  # 예측한 발전량

            predictions.append(predicted)  # 예측 결과 저장

            # 슬라이딩 윈도우: 새로운 예측을 다음 입력으로 포함
            current_sequence = np.vstack([current_sequence[1:], [test_data[features].iloc[seq_length + step].values]])
            current_sequence[-1, -1] = predicted  # 마지막 시퀀스의 타겟 값을 예측값으로 업데이트

    return predictions

# 2023-10-15 첫 24시간 시퀀스를 가져와서 예측 시작
initial_sequence = test_data[features].iloc[:seq_length].values

# 24시간 발전량 예측 수행 (슬라이딩 윈도우 방식)
forecast_steps = 24
predicted_values = recursive_forecast(model, initial_sequence, forecast_steps)

# 예측값 스케일 복원 (MinMaxScaler 역변환)
predicted_values = scaler_y.inverse_transform(np.array(predicted_values).reshape(-1, 1))

# 예측 결과 출력
print("Predicted values for 2023-10-15: ", predicted_values.flatten())

# 시각화
import plotly.graph_objects as go

true_values = test_data[target].values[:forecast_steps]  # 실제 값
predicted_values = predicted_values.flatten()  # 예측 값

fig = go.Figure()

# 실제 값 플롯
fig.add_trace(go.Scatter(x=test_data['time'].values[:forecast_steps], y=true_values, mode='lines', name='True Values'))

# 예측 값 플롯
fig.add_trace(go.Scatter(x=test_data['time'].values[:forecast_steps], y=predicted_values, mode='lines', name='Predicted Values'))

fig.update_layout(title='2023-10-15 발전량 예측 (Recursive Multi-step Forecast)',
                  xaxis_title='시간',
                  yaxis_title='발전량')

fig.show()


Epoch [10/50], Loss: 0.0116
Epoch [20/50], Loss: 0.0008
Epoch [30/50], Loss: 0.0033
Epoch [40/50], Loss: 0.0043
Epoch [50/50], Loss: 0.0030


IndexError: single positional indexer is out-of-bounds

### 2023-10-15 발전량 예측값을 predicted_amounts에 저장하고 시각화를 실행해주세요.

In [ ]:
# 시각화
fig = go.Figure()

# 실제 발전량 시각화
fig.add_trace(go.Scatter(x=data['time'], y=data['amount'],
                         mode='lines', name='Actual Amount'))

# 예측된 발전량 시각화 (2023-10-15의 24시간 동안의 예측)
fig.add_trace(go.Scatter(x=test_data['time'], y=predicted_amounts,
                         mode='lines', name='Predicted Amount', line=dict(dash='dot', color='red')))

# 레이아웃 설정
fig.update_layout(title='2023-10-15 24시간 발전량 예측',
                  xaxis_title='시간',
                  yaxis_title='발전량 (kWh)')

# 그래프 출력
fig.show()

### 정답을 적어주세요.

In [ ]:
# 예측된 24시간 발전량 출력
print(f'2023-10-15 예측 발전량 (24시간): {predicted_amounts} kWh')

---

### (필수) Informer모델은 transformer 모델의 어떤 부분을 개선하고자 했나요? (차이점을 중심으로 서술)

### (필수) 모델링 해석

In [ ]:
# 모델을 선택했다면 왜 선택했는지 본인만의 근거를 정리해주세요.
# 더 나아가 파라미터 선택의 기준이 있었다면 좋습니다.

### (선택) 데이터 해석

In [ ]:
# 데이터셋을 보고 느낀 생각이나 본인만의 논리 전개 방식을 정리해주세요.
# 전처리를 했다면 해당 전처리를 왜 했는지, 파생변수를 생성했다면 왜 만들었는지 등